In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torch import optim
import sklearn
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
import pandas as pd
import glob
import os

# Set the directory path
dir_path = r'D:\Study Material\4-2\Dataset_final\VeReMi_0_3600_2022-9-11_12.51.1'

# Get a list of all JSON files in the specified directory and its subdirectories
json_files = glob.glob(os.path.join(dir_path, '**', '*.json'), recursive=True)

# Create an empty dataframe to store the merged data
merged_df = pd.DataFrame()

# Loop through each file and read it into a pandas dataframe
for f in json_files:
    # Read the json file into a dataframe
    df = pd.read_json(f, orient='records', lines=True)[["pos", "spd"]]

    # Extract the label from the file prefix
    filename = os.path.basename(f)

    if filename.startswith('traceJSON'):
        label = filename.split('-')[3][1:]
    
        # Add the label column to the dataframe
        if int(label) == 0:
            df['label'] = 0
        elif int(label) >=1 and int(label) <= 8:
            df['label'] = 1
        else:
            df['label'] = 2
        

        # Append the dataframe to the merged dataframe
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# Now merged_df contains all the data from the JSON files, with label column added

In [ ]:
merged_df

In [ ]:
split = pd.DataFrame(merged_df['pos'].to_list(), columns = ['posX', 'posY','posZ'])
split = split.drop('posZ',axis=1)
merged_df = pd.concat([merged_df, split], axis=1) 
merged_df = merged_df.drop('pos',axis=1)

In [ ]:
split = pd.DataFrame(merged_df['spd'].to_list(), columns = ['spdX', 'spdY','spdZ'])
split = split.drop('spdZ',axis=1)
merged_df = pd.concat([merged_df, split], axis=1) 
merged_df = merged_df.drop('spd',axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

columns_to_scale = merged_df.columns.drop('label')

merged_df[columns_to_scale] = StandardScaler().fit_transform(merged_df[columns_to_scale])

In [ ]:
merged_df

In [ ]:
df_new = pd.DataFrame()
df_new = pd.concat([df_new, merged_df.loc[merged_df['label']==0].sample(frac=0.3)], ignore_index=True)
df_new = pd.concat([df_new, merged_df.loc[merged_df['label']==1]], ignore_index=True)
df_new = pd.concat([df_new, merged_df.loc[merged_df['label']==2]], ignore_index=True)
df_new.loc[df_new['label']==1]

In [ ]:
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size,50)
        self.fc2 = nn.Linear(50,30)
        self.fc3 = nn.Linear(30,20)
        self.fc4 = nn.Linear(20,num_classes)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#hyperparameters
input_size = 4
num_classes = 20
learning_rate = 0.001
batch_size = 64
num_epochs = 300

In [ ]:
#load data
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe[['posX', 'posY', 'spdX', 'spdY']].values
        self.targets = dataframe['label'].values

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        data = torch.tensor(self.data[index], dtype=torch.float)
        target = torch.tensor(self.targets[index], dtype=torch.long)
        return data, target

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
merged_df['label'] = le.fit_transform(merged_df['label'])

In [ ]:
# split dataframe into train and test sets
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

In [ ]:
# create dataset and dataloader
train_dataset = CustomDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#initialse network
model = NN(input_size=input_size,num_classes=num_classes).to(device)

In [ ]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
#train network
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
#         if batch_idx % 100 == 0:
        print('In batch: ', batch_idx)
        data = data.to(device=device)
        target = target.to(device=device)
        #get to correct shape
        data = data.reshape(data.shape[0],-1)
        #forward
        scores = model(data)
        loss = criterion(scores,target)
        #backward
        optimizer.zero_grad()
        loss.backward()
        #gradient descent
        optimizer.step()

In [ ]:
#check accuracy
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    y_list = []
    predictions_list = []
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0],-1)
#             print(y.shape)
            y_list.extend(y.tolist())
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            predictions_list.extend(predictions.tolist())
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
#     print(len(y_list))
#     print(len(predictions_list))
    print(classification_report(y_list,predictions_list))
    print("Confusion Matrix")
    print(confusion_matrix(y_list,predictions_list))
    model.train()

In [ ]:
check_accuracy(train_loader,model)

In [ ]:
check_accuracy(test_loader,model)

In [ ]:
# torch.save(model.state_dict(), 'saved_models\curr_model')

In [ ]:
model = NN(input_size=input_size,num_classes=num_classes).to(device)
# model.load_state_dict(torch.load('saved_models\curr_model'))
model.eval()

In [ ]:
# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=1)
        self.fc = nn.Linear(32 * (input_size // 4) * (input_size // 4), num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * (input_size // 4) * (input_size // 4))
        x = self.fc(x)
        return x

In [ ]:
model = CNN(input_size=input_size,num_classes=num_classes).to(device)

In [ ]:
#train network
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx % 1000 == 0:
            print('In batch: ', batch_idx)
        data = data.to(device=device)
        target = target.to(device=device)
        #get to correct shape
        data = data.reshape(data.shape[0],1,-1,1)
        #forward
        scores = model(data)
        loss = criterion(scores,target)
        #backward
        optimizer.zero_grad()
        loss.backward()
        #gradient descent
        optimizer.step()